# Project - Prediction (Live Prediction)

**Importing Libraries**

In [1]:
import pandas as pd
import numpy as np

**Importing Datasets**

In [2]:
file1 = pd.read_csv("IPL_Final_Data.csv")
file2 = pd.read_csv("IPL_Ball_By_Ball_Updated.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'IPL_Final_Data.csv'

### Data Preprocessing

In [ ]:
#Giving some common names to same names
file2['BattingTeam'] = file2['BattingTeam'].str.replace('Delhi Daredevils','Delhi Capitals')
file2['BattingTeam'] = file2['BattingTeam'].str.replace('Gujarat Lions','Gujarat Titans')
file2['BattingTeam'] = file2['BattingTeam'].str.replace('Kings XI Punjab','Punjab Kings')
file2['BattingTeam'] = file2['BattingTeam'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
file2.loc[file2['BattingTeam'].isin(['Rising Pune Supergiant','Rising Pune Supergiants']),'BattingTeam'] = 'Rising Pune Supergiants'

In [ ]:
#Giving some common names to same names
file2['BowlingTeam'] = file2['BowlingTeam'].str.replace('Delhi Daredevils','Delhi Capitals')
file2['BowlingTeam'] = file2['BowlingTeam'].str.replace('Gujarat Lions','Gujarat Titans')
file2['BowlingTeam'] = file2['BowlingTeam'].str.replace('Kings XI Punjab','Punjab Kings')
file2['BowlingTeam'] = file2['BowlingTeam'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
file2.loc[file2['BowlingTeam'].isin(['Rising Pune Supergiant','Rising Pune Supergiants']),'BowlingTeam'] = 'Rising Pune Supergiants'

In [ ]:
total_score = file2.groupby(['ID','innings']).sum()['total_run'].reset_index()

In [ ]:
total_score_first = total_score[total_score['innings']==1]

In [ ]:
match = pd.merge(file1,total_score_first[['ID','total_run']],left_on='ID',right_on='ID',how='left')
match.rename({'total_run':'FirstInningsRuns'},axis=1,inplace=True)
match.columns

In [ ]:
match = match[['ID', 'Team1', 'Team2', 'City', 'TossWinner','TossDecision',
       'WinningTeam','HomeGround','FirstInningsRuns']]

In [ ]:
delivery = match.merge(file2,on='ID')

In [ ]:
def replace_homeground(row):
    if row['HomeGround'] == 'Team1':
        return row['Team1']
    elif row['HomeGround'] == 'Team2':
        return row['Team2']
    else:
        return row['HomeGround']

In [ ]:
delivery['HomeGround'] = delivery.apply(replace_homeground, axis=1)
delivery

In [ ]:
delivery_second = delivery[delivery['innings']==2]
delivery_second

In [ ]:
delivery_second['CurrentScore'] = delivery_second.groupby('ID')['total_run'].cumsum()
delivery_second

In [ ]:
delivery_second['RunsLeft'] = delivery_second['FirstInningsRuns'] + 1 - delivery_second['CurrentScore'] 
delivery_second

In [ ]:
delivery_second['BallsLeft'] = 120 - (delivery_second['overs']*6 + delivery_second['ballnumber'])
delivery_second

In [ ]:
delivery_second['isWicketDelivery'] = delivery_second['isWicketDelivery'].astype(int)
wickets = delivery_second.groupby('ID')['isWicketDelivery'].cumsum().values
delivery_second['WicketsLeft'] = 10 - wickets
delivery_second

In [ ]:
delivery_second['CurrentRunRate'] = (delivery_second['CurrentScore']*6)/(120-delivery_second['BallsLeft'])
delivery_second

In [ ]:
delivery_second['RequiredRunRate'] = (delivery_second['RunsLeft']*6)/delivery_second['BallsLeft']
delivery_second

In [ ]:
def result(row):
    return 1 if row['BattingTeam'] == row['WinningTeam'] else 0

In [ ]:
delivery_second['Result'] = delivery_second.apply(result,axis=1)
delivery_second

In [ ]:
delivery_second.columns

In [ ]:
final_second = delivery_second[['City','HomeGround','BattingTeam','BowlingTeam','FirstInningsRuns', 'CurrentScore', 'RunsLeft',
       'BallsLeft', 'WicketsLeft', 'CurrentRunRate',
       'RequiredRunRate', 'Result']]
final_second

In [ ]:
def correct_homeground(row):
    if row['HomeGround'] == row['BattingTeam']:
        return 'Batting'
    elif row['HomeGround'] == row['BowlingTeam']:
        return 'Bowling'
    else:
        return 'Neutral'

In [ ]:
final_second['HomeGround'] = final_second.apply(correct_homeground, axis=1)
final_second

In [ ]:
final_second.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
final_second.dropna(inplace=True)

In [ ]:
columns = final_second.columns.tolist()
if 'Result' in columns:
    columns.remove('Result')
columns

### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

encode_columns = ['City','BattingTeam','BowlingTeam','HomeGround']
                  
for column in encode_columns:
                  final_second[column] = label_encoder.fit_transform(final_second[column])

final_second

### Train - Test Split

In [ ]:
X = final_second.drop(columns=['Result'])
y = final_second['Result']

In [ ]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=500)

### Model Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = LogisticRegression()
#model = RandomForestClassifier()
#model = DecisionTreeClassifier(random_state=42)
#model = SVC(kernel='linear', C=1)
#model = XGBClassifier()
#model = GaussianNB()
#model = SVC(kernel='rbf', random_state=0)
#model = KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)

### Model Fitting

In [ ]:
model.fit(X_train,y_train)

### Prediction

In [ ]:
y_pred = model.predict(X_test)

### Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test,y_pred)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [ ]:
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(classification_rep)

### Store Model

In [ ]:
import pickle
pickle.dump(model,open('D:\Shyam Mashru\Projects Main\Prediction\pipe2.pkl','wb'))